# Multivariate CNN Models - Multiple Parallel Series (Multiple-output)

In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


#### split a multivariate sequence into samples

In [2]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

## 1. Data preparation

#### define input sequence

In [3]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

#### convert to [rows, columns] structure

In [4]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

#### horizontally stack column

In [5]:
dataset = np.hstack((in_seq1, in_seq2, out_seq))

#### choose a number of time steps

In [6]:
n_steps = 3

#### convert into input/output

In [7]:
X, y = split_sequences(dataset, n_steps)

#### the dataset knows the number of features, e.g. 2

In [8]:
n_features = X.shape[2]

#### separate output

In [9]:
y1 = y[:, 0].reshape((y.shape[0], 1))
y2 = y[:, 1].reshape((y.shape[0], 1))
y3 = y[:, 2].reshape((y.shape[0], 1))

## 2. CNN Model

#### define model

In [10]:
visible = Input(shape=(n_steps, n_features))
cnn = Conv1D(filters=64, kernel_size=2, activation='relu')(visible)
cnn = MaxPooling1D(pool_size=2)(cnn)
cnn = Flatten()(cnn)
cnn = Dense(50, activation='relu')(cnn)

#### define outputs 1,2,3

In [11]:
output1 = Dense(1)(cnn)
output2 = Dense(1)(cnn)
output3 = Dense(1)(cnn)

#### tie together

In [12]:
model = Model(inputs=visible, outputs=[output1, output2, output3])
model.compile(optimizer='adam', loss='mse')

#### fit model

In [13]:
model.fit(X, [y1,y2,y3], epochs=2000, verbose=0)

#### demonstrate prediction

In [14]:
x_input = np.array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[array([[101.2034]], dtype=float32), array([[107.01904]], dtype=float32), array([[208.13991]], dtype=float32)]
